# Teleporting an Unknown Quantum State via Dual Einstein-Podolsky-Rosen Channels

*Physical Review Letters, 70:13, 1895. 29 March 1993*

https://researcher.watson.ibm.com/researcher/files/us-bennetc/BBCJPW.pdf.

Particle One is in an unknown state. So as not to disturb it, Alice has created an Einstein-Podolsky-Rosen pair of entangled particles (particles two and three). She sends particle three to Bob. She also measures particles one and two and sends the result of her measurement to Bob. Bob examines particle three and, in the light of the measurement information he received, figures out what the state of particle one was.

In [34]:
import qiskit
from qiskit import QuantumCircuit, transpile, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor

import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')

### Utilities to show multiparticle state vectors

In [35]:
# Two particles
def show_statevector(statevector):
    print(f'statevector\nuu, du, ud, dd:\n{np.around(statevector[0],3)} {np.around(statevector[1],3)} \
     {np.around(statevector[2], 3)} {np.around(statevector[3],3)}')
def show_statevector3(statevector, title=''):
    #Three particles
    print(f'title\
    uuu:{np.around(statevector[0],3)} duu{np.around(statevector[1],3)}\
     udu:{np.around(statevector[2], 3)} ddu{np.around(statevector[3],3)}\
     uud:{np.around(statevector[4], 3)} dud:{np.around(statevector[5],3)}\
     udd:{np.around(statevector[6], 3)} ddd{np.around(statevector[7], 3)}')


$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

#### zero=$\ket{u}$=(1,0) 
#### one=$\ket{d}$=(0,1)

## Bell Measurement Operator States

$\Psi^+=\sqrt{\frac{1}{2}}(\ket{ud}+\ket{du}=(1,0,0,0))$

$\Psi^-=\sqrt{\frac{1}{2}}(\ket{ud}-\ket{du}=(0,1,0,0))$

$\ket{\Phi^+}=\sqrt{\frac{1}{2}}(\ket{uu}+\ket{dd}=(0,0,1,0)$

$\ket{\Phi^-}=\sqrt{\frac{1}{2}}(\ket{uu}-\ket{dd}=(0,0,0,1)$

## Unitary Matrix for Measurement

$\bra{\Psi^+}\ket{uu}=0$

$\bra{\Psi^+}\ket{ud}= \sqrt{\frac{1}{2}}$

$\bra{\Psi^+}\ket{ud}= \sqrt{\frac{1}{2}}$

$\bra{\Psi^+}\ket{dd}=0$

$\bra{\Psi^-}\ket{uu}=0$

$\bra{\Psi^-}\ket{ud}=\sqrt{\frac{1}{2}}$

$\bra{\Psi^-}\ket{du}=\sqrt{\frac{1}{2}}$

$\bra{\Psi^-}\ket{dd}=0$

$\bra{\Phi^+}\ket{uu}=\sqrt{\frac{1}{2}}$

$\bra{\Phi^+}\ket{ud}=0$

$\bra{\Phi^+}\ket{du}=0$

$\bra{\Phi^+}\ket{dd}=\sqrt{\frac{1}{2}}$

$\bra{\Phi^-}\ket{uu}=\sqrt{\frac{1}{2}}$

$\bra{\Phi^-}\ket{ud}=0$

$\bra{\Phi^-}\ket{du}=0$

$\bra{\Phi^-}\ket{dd}=\sqrt{\frac{1}{2}}$

$U=\sqrt{\frac{1}{2}}\begin{bmatrix}
0&1&1&0\\
0&1&-1&0\\
1&0&0&1\\
1&0&0&-1\end{bmatrix}$



In [36]:
def measurement_gate(show=False):
    u=np.array([[0,1,1,0],\
                 [0,1,-1,0],\
                 [1,0,0,1],\
                  [1,0,0,-1]])/np.sqrt(2)

    if show:
        print('unitary matrix for projection')
        print(u[0,0],u[0,1], u[0,2], u[0,3])
        print(u[1,0],u[1,1], u[1,2], u[1,3])
        print(u[2,0], u[2,1], u[2,2], u[2,3])
        print(u[3,0], u[3,1], u[3,2], u[3,3])

    return u    

In [37]:

def make_psi_minus(qaa,qbb):
    qa = qaa-1
    qb=3-qbb-1
    circuit=qiskit.QuantumCircuit(3,3)
    circuit.h(qa)
    circuit.cx(qa,qb)
    circuit.x(qa)
    circuit.z(qa)
    return circuit


In [38]:
circuit=make_psi_minus(2,3)
circuit.draw()

q_0: ────────────────────
     ┌───┐     ┌───┐┌───┐
q_1: ┤ H ├──■──┤ X ├┤ Z ├
     └───┘┌─┴─┐└───┘└───┘
q_2: ─────┤ X ├──────────
          └───┘          
c: 3/════════════════════

In [39]:
# Debugging utilities
def helper(statevector):
    j=-1
    for i, a in enumerate(statevector):
        if np.abs(a)>.9:
            j=i
            break
    if j == 0:
        return 'Psi_plus'
    elif j == 1:
        return 'Psi_minus'
    elif j == 2:
        return 'Phi_plus'
    elif j == 3:
        return 'Phi_minus'
    elif j == 4:
        return 'Psi_plus'
    elif j == 5:
        return 'Psi_minus'
    elif j == 6:
        return 'Phi_plus'
    return 'Phi_minus'
def helper2(counts):
    c = counts[2]+counts[1]
    if c == '00':
        measurement = 'Psi_plus'
    elif c=='01': #ud
        measurement = 'Phi_plus'
    elif c=='10': #du
        measurement = 'Psi_minus'
    else:
        measurement = 'Phi__minus'
    return measurement

## Run Circuit with Particle One in Pure State

In [40]:
circuit = make_psi_minus(2,3)
circuit.h(0)
circuit.unitary(measurement_gate(),[0,1])
circuit.measure(0,0)
circuit.measure(1,1)
circuit.measure(2,2)
simulator=QasmSimulator()
simulator = qiskit.Aer.get_backend('statevector_simulator')

result = execute(circuit, backend=simulator, shots=1).result() 
statevector = result.get_statevector()
show_statevector3(statevector, title = helper(statevector))
counts=result.get_counts()
counts = list(counts.keys())[0]

title    uuu:(-0+0j) duu0j     udu:(-0+0j) ddu0j     uud:0j dud:0j     udd:(1+0j) ddd0j


## Alice Measures Particles One and Two

### Three-particle state
$\Psi_1 = (a,b)$

$\ket{\Psi_{123}}=\frac{1}{2}[\ket{\psi_{12}^-}(-a\ket{u_3}-b\ket{d_3})+$
$\ket{\Psi_{12}^+}(-a\ket{u_3}+b\ket{d_3}+$
$\ket{\Phi_{12}^-}(a\ket{d_3}+b\ket{u_3})+$
$\ket{\Phi_{12}+}(a\ket{d_3}-b\ket{u_3})]$

In [41]:
measurement = helper2(counts)
if measurement in ['Psi_plus', 'Psi_minus']:
    if counts[0]=='0':
        print('Particle three was "up" - particle one was "up"')
    else:
        print('Particle three was "down" - paricle three was down')
else:
    if counts[0]=='0':
        print('Particle three was "up" - particle one was "down"'),
    else:
        print('Particle three was "down" - particle one was "up"')

Particle three was "down" - particle one was "up"


In [42]:
#Helper functions for debugging
def helper4(p):
    if p == '0':
        return 'u'
    return 'd'

## Alice Measure Particles One and Two

In [43]:
def helper5(k):
    c1 = k[2]
    c2 = k[1]
    c = c1 + c2
    if c == '00':
        measurement = 'Psi_plus'
    elif c=='01':
        measurement = 'Phi_plus'
    elif c=='10':
        measurement = 'Psi_minus'
    else:
        measurement = 'Phi__minus'
    return measurement

### Bob Uses Results of Alice's Measurement to Interpret His Measurements on Particle 3

In [44]:
def count_particle_1(counts):
    p1_up=0
    p1_down=0
    for k in counts.keys():
        v = counts[k]
        
        measurement = helper5(k)

        p1=k[2]
        p2=k[1]
        p3=k[0]

        if measurement in ['Psi_plus', 'Psi_minus']:
            if p3=='0':
                p1_up += v
#                 print('Particle three was "up" - particle one was "up"')
            else:
                p1_down += v
#                 print('Particle three was "down" - paricle three was down')
        else:
            if p3 =='0':
                p1_down += v
#                 print('Particle three was "up" - particle one was "down"')
            else:
#                 print('Particle three was "down" - particle one was "up"')
                p1_up += v
        
#         print(measurement + ' ' + helper4(p1)+helper4(p2)+helper4(p3)+ ' ' + str(v))
    percentage = np.around(p1_up/(p1_up+p1_down)*100,0)
    print(f'Particle 1 was up {percentage}%')

## Start With Particle One in Mixed State

In [45]:
circuit = make_psi_minus(2,3)
circuit.h(0)
circuit.unitary(measurement_gate(),[0,1])
circuit.measure(0,0)
circuit.measure(1,1)
circuit.measure(2,2)
simulator=QasmSimulator()

result = execute(circuit, backend=simulator, shots=1024).result() 
counts=result.get_counts()
count_particle_1(counts)

Particle 1 was up 49.0%
